### Query Enhancement – Query Expansion Techniques

In a RAG pipeline, the quality of the query sent to the retriever determines how good the retrieved context is — and therefore, how accurate the LLM’s final answer will be.

That’s where Query Expansion / Enhancement comes in.

#### 🎯 What is Query Enhancement?
Query enhancement refers to techniques used to improve or reformulate the user query to retrieve better, more relevant documents from the knowledge base.
It is especially useful when:

- The original query is short, ambiguous, or under-specified
- You want to broaden the scope to catch synonyms, related phrases, or spelling variants

In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableMap

c:\Users\achra\AppData\Local\anaconda3\envs\RAG_project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## step1 : Load and split the dataset
loader = TextLoader("data/langchain_crewai_dataset.txt")
raw_docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(raw_docs)


In [3]:
chunks

[Document(metadata={'source': 'data/langchain_crewai_dataset.txt'}, page_content='LangChain is an open-source framework designed for developing applications powered by large language models (LLMs). It simplifies the process of building, managing, and scaling complex chains of thought by abstracting prompt management, retrieval, memory, and agent orchestration. Developers can use'),
 Document(metadata={'source': 'data/langchain_crewai_dataset.txt'}, page_content='and agent orchestration. Developers can use LangChain to create end-to-end pipelines that connect LLMs with tools, APIs, vector databases, and other knowledge sources. (v1)'),
 Document(metadata={'source': 'data/langchain_crewai_dataset.txt'}, page_content='At the heart of LangChain lies the concept of chains, which are sequences of calls to LLMs and other tools. Chains can be simple, such as a single prompt fed to an LLM, or complex, involving multiple conditionally executed steps. LangChain makes it easy to compose and reuse 

In [4]:
### step 2: Vector Store
embedding_model=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore=FAISS.from_documents(chunks,embedding_model)

## step 3:MMR Retriever
retriever=vectorstore.as_retriever(search_type="mmr",search_kwargs={"k":5})
retriever


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023557CD25E0>, search_type='mmr', search_kwargs={'k': 5})

In [5]:
## step 4 : LLM and Prompt

import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

llm=init_chat_model("openai:o4-mini")
llm


ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002355B2B8A30>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002355B315D30>, root_client=<openai.OpenAI object at 0x000002355B2B8A90>, root_async_client=<openai.AsyncOpenAI object at 0x0000023548F400D0>, model_name='o4-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [6]:
# Query expansion
query_expansion_prompt = PromptTemplate.from_template("""
You are a helpful assistant. Expand the following query to improve document retrieval by adding relevant synonyms, technical terms, and useful context.

Original query: "{query}"

Expanded query:
""")

query_expansion_chain=query_expansion_prompt| llm | StrOutputParser()
query_expansion_chain

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='\nYou are a helpful assistant. Expand the following query to improve document retrieval by adding relevant synonyms, technical terms, and useful context.\n\nOriginal query: "{query}"\n\nExpanded query:\n')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002355B2B8A30>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002355B315D30>, root_client=<openai.OpenAI object at 0x000002355B2B8A90>, root_async_client=<openai.AsyncOpenAI object at 0x0000023548F400D0>, model_name='o4-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)
| StrOutputParser()

In [7]:
query_expansion_chain.invoke({"query":"Langchain memory"})

'Expanded query:\n\n(LangChain OR "LangChain SDK") AND (memory OR "memory module" OR "memory management" OR "stateful memory" OR "session memory" OR "persistent memory" OR "context memory" OR "conversational memory" OR "conversation history" OR "context retention" OR "memory buffer" OR "memory retriever") AND ("retrieval-augmented generation" OR RAG OR "vector store" OR embeddings OR Pinecone OR FAISS OR Chroma OR Weaviate OR Redis) AND (LLM OR "large language model" OR GPT-3.5 OR GPT-4 OR "chain-of-thought" OR prompt)'

In [8]:
# RAG answering prompt
answer_prompt = PromptTemplate.from_template("""
Answer the question based on the context below.

Context:
{context}

Question: {input}
""")

document_chain=create_stuff_documents_chain(llm=llm,prompt=answer_prompt)

In [9]:
# Step 5: Full RAG pipeline with query expansion
rag_pipeline = (
    RunnableMap({
        "input": lambda x: x["input"],
        "context": lambda x: retriever.invoke(query_expansion_chain.invoke({"query": x["input"]}))
    })
    | document_chain
)

In [10]:
# Step 6: Run query
query = {"input": "What types of memory does LangChain support?"}
print(query_expansion_chain.invoke({"query":query}))
response = rag_pipeline.invoke(query)
print("✅ Answer:\n", response)

Expanded query:  
“What types of memory mechanisms and modules does LangChain support for managing and persisting conversational state in LLM applications? For example, what short-term and long-term memory architectures (buffer memory, windowed buffer, summarized memory), memory backends (in-memory, key–value stores, SQL/relational, document stores), and vector‐store or embedding‐based memory solutions (FAISS, Pinecone, Redis, Chroma, Weaviate, Milvus) are available? How do these memory components—such as ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryMemory, RedisMemory, SQLAlchemyMemory, and custom memory classes—fit into LangChain chains, agents, and chatbots to enable retrieval-augmented generation, context retention, and stateful dialogue?”
✅ Answer:
 LangChain provides plug-and-play memory modules, most notably:  
- ConversationBufferMemory – keeps a running buffer of the full chat history  
- ConversationSummaryMemory – continually summarizes earlie

In [13]:
# Step 6: Run query
query = {"input": "CrewAI agents?"}
print(query_expansion_chain.invoke({"query":query}))
response = rag_pipeline.invoke(query)
print("✅ Answer:\n", response)

Expanded query:  
("CrewAI agents" OR "Crew AI agents" OR "AI crew agents" OR "autonomous crew agents" OR "digital crew assistants" OR "virtual crew members" OR "crew management bots" OR "software agents for crew management")  
AND (AI OR "artificial intelligence" OR "machine learning" OR "deep learning" OR "reinforcement learning")  
AND ("multi-agent system" OR "agent-based modeling" OR "autonomous software agent" OR "agent architecture")  
AND (crew OR workforce OR staffing OR "resource management" OR scheduling OR logistics OR "task allocation" OR operations)  
AND (maritime OR aviation OR hospitality OR "field service" OR manufacturing OR transportation OR healthcare)
✅ Answer:
 CrewAI agents are semi-autonomous AI “workers” organized into a structured workflow.  Each agent is assigned a specific role—for example:  
  • Researcher: gathers data, does background digging  
  • Planner: devises the overall strategy or task plan  
  • Executor: carries out the concrete steps (API call